<a href="https://colab.research.google.com/github/rudyhendrawn/traditional-dance-video-classification/blob/main/i3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rfv sample_data

### Mounting Repository

Mount the repository to get all the data (directories and utilities files) ready to use.

In [1]:
!git clone https://github.com/rudyhendrawn/traditional-dance-video-classification.git
!mv traditional-dance-video-classification/* .
!rm -rfv traditional-dance-video-classification

Cloning into 'traditional-dance-video-classification'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 184 (delta 58), reused 66 (delta 16), pack-reused 47
Receiving objects: 100% (184/184), 127.21 MiB | 26.52 MiB/s, done.
Resolving deltas: 100% (71/71), done.
mv: cannot move 'traditional-dance-video-classification/checkpoint' to './checkpoint': Directory not empty
mv: cannot move 'traditional-dance-video-classification/dataset' to './dataset': Directory not empty
mv: cannot move 'traditional-dance-video-classification/history' to './history': Directory not empty
mv: cannot move 'traditional-dance-video-classification/img' to './img': Directory not empty
mv: cannot move 'traditional-dance-video-classification/lib' to './lib': Directory not empty
mv: cannot move 'traditional-dance-video-classification/misc' to './misc': Directory not empty
mv: cannot move 'traditional-dance-video-c

### Mounting Google Drive

Google drive need to be setup and mounted to this specific project. Using this code below to setup and mount the google drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Initial Setup 🧑‍💻

Setup the project, import the required dependencies

> Make sure the `lib` directory exists and include: `keras_video`, `i3d_inception.py`

In [3]:
import os
import time
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import lib.helpers as helpers

from lib.keras_video import VideoFrameGenerator
from lib.i3d_inception import Inception_Inflated3d

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from IPython import get_ipython

get_ipython().run_line_magic("matplotlib", "inline")
warnings.filterwarnings("ignore")

In [4]:
DS = os.path.sep
DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/Tari-Bali' # Change with the correct path to your dataset

# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile(os.path.join(DATASET_PATH, 'Dasar-Gerakan-Tari-Bali-All-Men.zip'), 'r') as zObject:
    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(path=os.path.join('/content'))

# Read the video from specified path
DATASET_DIR = os.path.join('/content/Dasar-Gerakan-Tari-Bali-All-Men')

### Generating Class Names & Glob Pattern

Load all the file paths at the DATASET_DIR to generate the class names. Also, define glob pattern to get the dataset

In [5]:
class_names = helpers.get_generated_class_names(DATASET_DIR, "train")
print(class_names)
train_glob_pattern = helpers.get_generated_glob_pattern(DATASET_DIR, "train")
test_glob_pattern = helpers.get_generated_glob_pattern(DATASET_DIR, "test");
val_glob_pattern = helpers.get_generated_glob_pattern(DATASET_DIR, "val");

['Agem_Kanan', 'Agem_Kiri', 'Gandang_Gandang', 'Malpal', 'Nayog', 'Nepuk_Kampuh', 'Oyod', 'Piles', 'Seledet', 'Tapak_Sirang_Pada', 'Ulap_Ulap']


### Dataset Setup

Setup the dataset with `keras_video.VideoFrameGenerator` to do the dataset extraction

In [6]:
BATCH_SIZE = 8
NB_COLOR_CHANNELS = 3
NB_FRAMES = 30
RESOLUTION = (224, 224)

train_dataset_generator = VideoFrameGenerator(
    batch_size=BATCH_SIZE,
    classes=class_names,
    glob_pattern=train_glob_pattern,
    nb_channel=NB_COLOR_CHANNELS,
    nb_frames=NB_FRAMES,
    seed=42,
    target_shape=RESOLUTION,
    transformation=None,
    use_frame_cache=False
)

Total data: 11 classes for 531 files for train


In [7]:
test_dataset_generator = VideoFrameGenerator(
    batch_size=BATCH_SIZE,
    classes=class_names,
    glob_pattern=test_glob_pattern,
    nb_channel=NB_COLOR_CHANNELS,
    nb_frames=NB_FRAMES,
    seed=42,
    target_shape=RESOLUTION,
    transformation=None,
    use_frame_cache=False
)

Total data: 11 classes for 165 files for train


In [8]:
val_dataset_generator = VideoFrameGenerator(
    batch_size=BATCH_SIZE,
    classes=class_names,
    glob_pattern=val_glob_pattern,
    nb_channel=NB_COLOR_CHANNELS,
    nb_frames=NB_FRAMES,
    seed=42,
    target_shape=RESOLUTION,
    transformation=None,
    use_frame_cache=False
)

Total data: 11 classes for 132 files for train


In [9]:
for x, y in train_dataset_generator:
    print(f"Data shape in data generator {x.shape}")
    break

for x, y in val_dataset_generator:
    print(f"Data shape in data generator {x.shape}")
    break

for x, y in test_dataset_generator:
    print(f"Data shape in data generator {x.shape}")
    break

Data shape in data generator (8, 30, 224, 224, 3)
Data shape in data generator (8, 30, 224, 224, 3)
Data shape in data generator (8, 30, 224, 224, 3)


In [10]:
input_shape = (NB_FRAMES,) + RESOLUTION + (NB_COLOR_CHANNELS,)
print(input_shape)

(30, 224, 224, 3)


### Prepare `i3d` Layer

Preparing the configuration to create the `i3d` layer to add to the created model

In [11]:
i3d_layer = Inception_Inflated3d(
    classes=3,
    include_top=False,
    input_shape=input_shape,
    weights="rgb_kinetics_only"
)

i3d_layer.trainable = False

49595336/49595336 [==============================] - 1s 0us/step


### Model Creation

Creating `Sequential` model and add `i3d` and some other layers to the created model

In [14]:
model = tf.keras.models.Sequential()

# classification layer
model.add(i3d_layer)
model.add(tf.keras.layers.GlobalAveragePooling3D())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(int(len(class_names)), activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 i3d_inception (Functional)  (None, 3, 1, 1, 1024)     12294544  
                                                                 
 global_average_pooling3d (G  (None, 1024)             0         
 lobalAveragePooling3D)                                          
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 11)                5643      
                                                                 
Total params: 12,824,987
Trainable params: 530,443
Non-trainable params: 12,294,544
______________________________________

### Compiling & Fitting Setup

Some setup configuration for compiling and fitting the model. Defining epochs, earlystopping, checkpoint, and callbacks.

In [15]:
epochs = 3
model_earlystopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoint/i3d-{epoch:02d}-{val_loss:.2f}.h5",
    mode="min",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

model_callbacks = [model_earlystopping, model_checkpoint]

### Model Compile

Compiling model with pre-defined configuration

In [16]:
model.compile(
    loss="categorical_crossentropy",
    metrics=["acc"],
    optimizer="adam"
)

### Model Training/Fitting

Fit the model with real dataset with defined epochs and callbacks

In [17]:
tf.keras.backend.clear_session()

start_time = time.time()

model_history = model.fit(
    train_dataset_generator,
    callbacks=model_callbacks,
    epochs=epochs,
    validation_data=val_dataset_generator
)

end_time = time.time()
exec_time = end_time - start_time

print("Fitting execution time : {}s".format(exec_time))

Epoch 1/3
66/66 [==============================] - ETA: 0s - loss: 2.4475 - acc: 0.1591
Epoch 1: val_loss improved from inf to 2.16803, saving model to checkpoint/i3d-01-2.17.h5
66/66 [==============================] - 179s 3s/step - loss: 2.4475 - acc: 0.1591 - val_loss: 2.1680 - val_acc: 0.2344
Epoch 2/3
66/66 [==============================] - ETA: 0s - loss: 2.1174 - acc: 0.2519
Epoch 2: val_loss improved from 2.16803 to 2.07173, saving model to checkpoint/i3d-02-2.07.h5
66/66 [==============================] - 165s 2s/step - loss: 2.1174 - acc: 0.2519 - val_loss: 2.0717 - val_acc: 0.1953
Epoch 3/3
66/66 [==============================] - ETA: 0s - loss: 2.0757 - acc: 0.2481
Epoch 3: val_loss improved from 2.07173 to 1.97965, saving model to checkpoint/i3d-03-1.98.h5
66/66 [==============================] - 179s 3s/step - loss: 2.0757 - acc: 0.2481 - val_loss: 1.9797 - val_acc: 0.2109
Fitting execution time : 524.8913578987122s


### Save Model

Saving model file into `model` directory

In [ ]:
# model.save("model/dance/i3d-5e.h5")

### Acc Visualization

Visualizing acc data with Matplotlib graph

In [ ]:
helpers.get_visualized_graph(
  plots=[model_history.history["acc"], model_history.history["val_acc"]],
  title="Model Accuracy",
  x_label="Epoch",
  y_label="Accuracy",
  legend=["train", "test"]
).show()

### Loss Visualization

Visualizing loss data with Matplotlib graph

In [ ]:
helpers.get_visualized_graph(
  plots=[model_history.history["loss"], model_history.history["val_loss"]],
  title="Model Loss",
  x_label="Epoch",
  y_label="Loss",
  legend=["train", "test"]
).show()

### Export Dataframe From Model

Export dataframe to `.csv` file from the model history via Pandas library

In [ ]:
model_history_dataframe = pd.DataFrame(model_history.history)
model_history_fpath = "history/dance/i3d-5e.csv"

with open(model_history_fpath, mode="w") as history_file:
    model_history_dataframe.to_csv(history_file)

### Model Evaluation

Evaluating model with test dataset

In [ ]:
model.evaluate(test_dataset_generator)

### Populate Y Data

Populating Y's `prediction` and `test` data with test dataset

In [ ]:
y_prediction_max, y_true = helpers.get_populated_y_data(
    batch_size=BATCH_SIZE,
    generator=test_dataset_generator,
    model=model
)

### Score Visualization

Visualizing some of calculated model score types, like `accuracy`, `precision`, `recall`, and `f1` score

In [ ]:
score_accuracy, score_precision, score_recall, score_f1 = helpers.get_calculated_score(y_true, y_prediction_max)

print(f"Accuracy Score\t: {np.round(score_accuracy, 3)}")
print(f"Precision Score\t: {np.round(score_precision, 3)}")
print(f"Recall Score\t: {np.round(score_recall, 3)}")
print(f"F1 Score\t: {np.round(score_f1, 3)}")

### Classification Report Visualization

Visualizing classification report of test dataset

In [ ]:
test_class_names = test_dataset_generator.classes

print(classification_report(
    y_true,
    y_prediction_max,
    target_names=test_class_names
))

### Confusion Matrix Visualization

Visualizing confusion matrix with heatmap table

In [ ]:
confusion_matrix_result = confusion_matrix(y_true, y_prediction_max)

sns.heatmap(
    confusion_matrix_result,
    annot=True,
    cmap="Blues"
)

### AUC Score Visualization

Visualization of AUC score calculated with FPR and TPR

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_prediction_max, pos_label=6)
score_auc = auc(fpr, tpr)

print(f"AUC Score\t: {np.round(score_auc, 3)}")

### True/False Positive Rate Visualization

Visualizing `true`/`false` rate with Matplotlib graph calculated from FPR and TPR

In [ ]:
plt.plot(fpr, tpr, marker=".")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.show()

### Visualizing Checkpoint Model

Visualizing all the score/calculated score from the checkpoint model

In [ ]:
from tensorflow.keras.models import load_model

checkpoint_model = load_model("checkpoint/i3d.h5")
checkpoint_model.evaluate(test_dataset_generator)

### Populate Y Data

Populating checkpoint model Y's `prediction` and `test` data with test dataset

In [ ]:
y_prediction_max, y_true = helpers.get_populated_y_data(
    batch_size=BATCH_SIZE,
    generator=test_dataset_generator,
    model=model
)

### Score Visualization

Visualizing some of calculated checkpoint model score types, like `accuracy`, `precision`, `recall`, and `f1` score

In [ ]:
score_accuracy, score_precision, score_recall, score_f1 = helpers.get_calculated_score(y_true, y_prediction_max)

print(f"Accuracy Score\t: {np.round(score_accuracy, 3)}")
print(f"Precision Score\t: {np.round(score_precision, 3)}")
print(f"Recall Score\t: {np.round(score_recall, 3)}")
print(f"F1 Score\t: {np.round(score_f1, 3)}")

### Classification Report Visualization

Visualizing checkpoint model classification report of test dataset

In [ ]:
test_class_names = test_dataset_generator.classes

print(classification_report(
    y_true,
    y_prediction_max,
    target_names=test_class_names
))

### Confusion Matrix Visualization

Visualizing checkpoint model confusion matrix with heatmap table

In [ ]:
confusion_matrix_result = confusion_matrix(y_true, y_prediction_max)

sns.heatmap(
    confusion_matrix_result,
    annot=True,
    cmap="Blues"
)

### AUC Score Visualization

Visualization of checkpoint model AUC score calculated with FPR and TPR

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_prediction_max, pos_label=6)
score_auc = auc(fpr, tpr)

print(f"AUC Score\t: {np.round(score_auc, 3)}")

### True/False Positive Rate Visualization

Visualizing checkpoint model `true`/`false` rate with Matplotlib graph calculated from FPR and TPR

In [ ]:
plt.plot(fpr, tpr, marker=".")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.show()